# TP3 : Inférence variationnelle dans le modèle *Latent Dirichlet Allocation*

## G3 SDI - Estimation Avancée

Dans ce dernier TP, on s'intéresse au modèle *Latent Dirichlet Allocation* ([Blei et al. (2003)](https://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf)), un célèbre modèle probabiliste pour données textuelles. Voir le .pdf joint à l'archive pour la description complète du modèle. Dans ce modèle, la loi a posteriori est intractable, et nous utiliserons l'approche variationnelle pour en trouver une approximation.

Using the notations from the .pdf file describing the model, the intractable posterior is :
$$p(\boldsymbol{\beta}, \boldsymbol{\theta} | \mathcal{D}),$$
which we are going to approximate in the following way :
$$\simeq \left[ \prod_{k=1}^K q(\boldsymbol{\beta_k}) \right] \left[ \prod_{d=1}^D q(\boldsymbol{\theta_d}) \right] , $$
with :
* $q(\boldsymbol{\beta_k})$ a Dirichlet distribution (of size V) with variational parameters $[\lambda_{k1}, ...,\lambda_{kV}]$ ;
* $q(\boldsymbol{\theta_d})$ a Dirichlet distribution (of size K) with variational parameters $[\gamma_{d1}, ...,\gamma_{dK}]$.

### Instructions

1. Renommez votre notebook sous la forme `tp3_Nom1_Nom2.ipynb`. 

2. Votre code, ainsi que toute sortie du code, doivent être commentés !

3. Déposez votre notebook sur Moodle dans la section prévue à cet effet avant la date limite.

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Compte-rendu écrit par [nom1], [nom2], date.
</div>

In [ ]:
# Import usual libraries
import numpy as np
from matplotlib import pyplot as plt

# Specific for this lab
from sklearn.decomposition import LatentDirichletAllocation

**Q0.** À quelle difficulté principale se heurterait un algorithme MCMC dans ce contexte ?

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Votre réponse ici
</div>

### Partie 1 - Préparation des données

On utilise le dataset `20newsgroups` (voir par exemple [ici](https://scikit-learn.org/stable/datasets/real_world.html#newsgroups-dataset)), qui contient des messages postés dans différentes catégories sur un forum dans les années 90.

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

# Load data
newsgroups = fetch_20newsgroups().data

**Q1.** En LDA, sous quelle forme est représentée un document ?

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Votre réponse ici
</div>

**Q2.** Créer la matrice des données en utilisant la fonction `CountVectorizer` en utilisant les arguments :
* `max_df` = 0.95 ;
* `max_features` = 1000 ;
* `stop_words` = "english".

Expliquer à quoi correspondent ces 3 arguments.

In [ ]:
#####
### YOUR CODE HERE
#####

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Votre réponse ici
</div>

**Q3.** Rappeler à quoi correspondent les dimensions de la matrice des données obtenues.

Calculer son pourcentage d'éléments non-nuls. Est-ce surprenant ?

In [ ]:
#####
### YOUR CODE HERE
#####

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Votre réponse ici
</div>

### Partie 2 - Latent Dirichlet Allocation (LDA)

**Q4.** Entraîner le modèle LDA avec les arguments suivants :
* `n_components` = 8 ;
* `learning_method` = "online" ;
* `max_iter` = 50 ;
* `doc_topic_prior` = 0.1 ;
* `topic_word_prior` = 0.1 ;
* `learning_offset` = 100.

Idem, expliquer à quoi correspondent chacun de ces arguments.

In [ ]:
#####
### YOUR CODE HERE
#####

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Votre réponse ici
</div>

**Q5.** Que contient l'attribut `.components` ?

Comment obtenir le MMSE de $\boldsymbol{\beta}$ à partir de cet attribut ? L'implémenter.

In [ ]:
#####
### YOUR CODE HERE
#####

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Votre réponse ici
</div>

**Q6.** À partir de $\hat{\boldsymbol{\beta}}_{MMSE}$, afficher les 10 mots les plus courants par topic. Pouvez-vous interpréter les topics obtenus ?

Si vous souhaitez rajouter certains mots à la liste des stop words pour améliorer l'interprétabilité, utilisez le bloc de code ci-dessous.

In [ ]:
#####
### YOUR CODE HERE
#####

In [ ]:
# To add additional stop words

from sklearn.feature_extraction import text 

my_additional_stop_words = ### YOUR LIST HERE ### #list of strings
my_stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Votre réponse ici
</div>

**Q7.** Que contient `lda.transform(X)` ?

Regarder si les documents sont en général plutôt associés à un ou plusieurs topics.

In [ ]:
#####
### YOUR CODE HERE
#####

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Votre réponse ici
</div>

### Partie 3 - Pour aller plus loin...

Répondre aux questions sans implémenter les solutions discutées.

**Q8.** Quelle est l'influence des paramètres $\alpha$ et $\eta$ ?

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Votre réponse ici
</div>

**Q9.** Pour aujourd'hui, le nombre de topics $K$ était fixé à l'avance. Comment pourrait-on apprendre ou choisir la valeur de $K$ ?

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Votre réponse ici
</div>

**Q10.** Quelles sont les principales limites du modèle LDA et quelles potentielles améliorations proposez-vous ?

<div style="background-color: rgba(255, 255, 0, 0.15); padding: 8px;">
Votre réponse ici
</div>